In [2]:
import pandas as pd

df=pd.read_csv("data/training_sample.csv")
df

,Unnamed: 0,id,img,label,text
0,0,53692,img/53692.png,1,"wait, mohammad come back!! i didn't mean to la..."
1,1,78469,img/78469.png,1,when each letter is a mental disorder
2,2,93512,img/93512.png,1,once you go black so does your eye
3,3,10829,img/10829.png,1,"its ok, let them in only a few will kill"
4,4,49536,img/49536.png,0,hillary clinton caught telling the truth! apri...
...,...,...,...,...,...
995,995,46283,img/46283.png,0,when you're dedicated to working out everyday
996,996,46359,img/46359.png,0,then something amazing started to happen! last...
997,997,58061,img/58061.png,1,i now have two identities transgender and tran...
998,998,36152,img/36152.png,0,when your eating out your grandma and you tast...


In [2]:
from src.config import Config
from models.classifiers import BinaryClassifier
import torch

config=Config()
model_test=BinaryClassifier(config)

model_test.load_state_dict(torch.load("models/checkpoints/bimodal_classifier.pth"))
model_test.eval()

BinaryClassifier(
  (model): CLIPModel(
    (text_model): CLIPTextTransformer(
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(77, 512)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-11): 12 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=512, out_features=2048, bias=True)
              (fc2): Linear(in_features=2048, out_features=512, bias=True

In [ ]:
from src.dataloader import MultimodalDataset
from torch.utils.data import DataLoader
from transformers import CLIPProcessor, CLIPModel
from src.config import BimodalConfig
import pandas as pd


processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
bimodal_config=BimodalConfig()
model=CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer=processor.tokenizer
test_df=pd.read_json("data/original_test.jsonl", lines=True)

test_dataset=MultimodalDataset(test_df, processor, bimodal_config.mode)
test_loader=DataLoader(test_dataset, batch_size=bimodal_config.batch_size,shuffle=True,num_workers=0)

first_batch=next(iter(test_loader))

datapoint = {
    "input_ids": first_batch["input_ids"][0],           # shape: (seq_len,)
    "attention_mask": first_batch["attention_mask"][0], # shape: (seq_len,)
    "pixel_values": first_batch["pixel_values"][0],     # shape: (C, H, W)
    "labels": first_batch["labels"][0]                  # scalar
}




dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'labels'])

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os



# Dummy attention matrix (e.g. 8x8)
attention_matrix = np.random.rand(8, 8)

# Dummy tokens
tokens = [f"tok_{i}" for i in range(8)]

# Create a grid of subplots for 8 attention heads
fig, axes = plt.subplots(2, 4, figsize=(16, 8))  # 2 rows x 4 cols

for i in range(8):
    ax = axes[i // 4, i % 4]  # Position in subplot grid

    sns.heatmap(attention_matrix, xticklabels=tokens, yticklabels=tokens,
                cmap="viridis", cbar=False, ax=ax)

    ax.set_title(f"Head {i}", fontsize=10)
    ax.tick_params(axis='x', rotation=90, labelsize=6)
    ax.tick_params(axis='y', labelsize=6)

# Adjust spacing between subplots
plt.tight_layout()

name="test"

# Save the figure
plt.savefig(os.path.join("interpretability/plots", name), bbox_inches='tight', dpi=300)
plt.close()

print("Saved to outputs/test_attention_heads.png")


Saved to outputs/test_attention_heads.png
